<a href="https://colab.research.google.com/github/R1A2H1L1/KaggleProject/blob/main/99%20-%20modelo%20soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
# Instalar XGBoost (si es necesario)
!pip install xgboost
!pip install Unidecode


# Importar librerías necesarias
import pandas as pd
import numpy as np
from unidecode import unidecode
import os
from google.colab import userdata

# Cargar datos
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# Verificar tamaños
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

Train shape: (692500, 21)
Test shape: (296786, 20)


In [43]:
user = userdata.get('KAGGLE_USERNAME')
key = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = user
os.environ["KAGGLE_KEY"] = key

In [44]:
def format_df(df):
    df = df.copy()
    df = df.fillna("Miss")

    # ===== One-hot encoding básico =====
    onehot_cols = ["F_TIENEINTERNET", "E_PAGOMATRICULAPROPIO"]
    for col in onehot_cols:
        dummies = pd.get_dummies(df[col], prefix=col)
        df = pd.concat([df.drop(col, axis=1), dummies], axis=1)

    # ===== Mapeos ordinales =====
    mappings = {
        "E_VALORMATRICULAUNIVERSIDAD": {
            'Miss': 0, 'No pagó matrícula': 1, 'Menos de 500 mil': 2,
            'Entre 500 mil y menos de 1 millón': 3, 'Entre 1 millón y menos de 2.5 millones': 4,
            'Entre 2.5 millones y menos de 4 millones': 5, 'Entre 4 millones y menos de 5.5 millones': 6,
            'Entre 5.5 millones y menos de 7 millones': 7, 'Más de 7 millones': 8
        },
        "E_HORASSEMANATRABAJA": {
            'Miss': 0, '0': 1, 'Menos de 10 horas': 2, 'Entre 11 y 20 horas': 3,
            'Entre 21 y 30 horas': 4, 'Más de 30 horas': 5
        },
        "F_ESTRATOVIVIENDA": {
            'Miss': -1, 'Sin Estrato': 0, 'Estrato 1': 1, 'Estrato 2': 2,
            'Estrato 3': 3, 'Estrato 4': 4, 'Estrato 5': 5, 'Estrato 6': 6
        },
        "F_EDUCACIONPADRE": {
            'Miss': -2, 'No Aplica': -1, 'Ninguno': 0, 'No sabe': 1,
            'Primaria incompleta': 2, 'Primaria completa': 3,
            'Secundaria (Bachillerato) incompleta': 4, 'Secundaria (Bachillerato) completa': 5,
            'Técnica o tecnológica incompleta': 6, 'Técnica o tecnológica completa': 7,
            'Educación profesional incompleta': 8, 'Educación profesional completa': 9,
            'Postgrado': 10
        },
        "F_EDUCACIONMADRE": {
            'Miss': -2, 'No Aplica': -1, 'Ninguno': 0, 'No sabe': 1,
            'Primaria incompleta': 2, 'Primaria completa': 3,
            'Secundaria (Bachillerato) incompleta': 4, 'Secundaria (Bachillerato) completa': 5,
            'Técnica o tecnológica incompleta': 6, 'Técnica o tecnológica completa': 7,
            'Educación profesional incompleta': 8, 'Educación profesional completa': 9,
            'Postgrado': 10
        },
        "RENDIMIENTO_GLOBAL": {
            'bajo': 1, 'medio-bajo': 2, 'medio-alto': 3, 'alto': 4
        }
    }

    for col, mapping in mappings.items():
        if col in df.columns:
            df[col] = df[col].map(mapping)

    # ===== Normalizar nombres de programas =====
    if "E_PRGM_ACADEMICO" in df.columns:
        df["E_PRGM_ACADEMICO"] = df["E_PRGM_ACADEMICO"].apply(lambda x: unidecode(str(x)).upper())
        freqs = df["E_PRGM_ACADEMICO"].value_counts()
        rare_programs = freqs[freqs < 10].index
        df["E_PRGM_ACADEMICO"] = df["E_PRGM_ACADEMICO"].replace(rare_programs, "OTROS")
        df["E_PRGM_ACADEMICO"] = pd.factorize(df["E_PRGM_ACADEMICO"])[0]

    # ===== Codificar departamento =====
    if "E_PRGM_DEPARTAMENTO" in df.columns:
        df["E_PRGM_DEPARTAMENTO"] = df["E_PRGM_DEPARTAMENTO"].apply(lambda x: unidecode(str(x)).upper())
        dummies = pd.get_dummies(df["E_PRGM_DEPARTAMENTO"], prefix="DEPTO")
        df = pd.concat([df.drop("E_PRGM_DEPARTAMENTO", axis=1), dummies], axis=1)

    # ===== 🚀 NUEVAS VARIABLES CREADAS =====
    df["TRABAJA_Y_NO_INTERNET"] = (
        (df["E_HORASSEMANATRABAJA"] > 1) &
        (df.get("F_TIENEINTERNET_No", 0) == 1)
    ).astype(int)

    df["PAGA_MATRICULA_BAJO_ESTRATO"] = (
        (df["E_VALORMATRICULAUNIVERSIDAD"] > 1) &
        (df["F_ESTRATOVIVIENDA"] <= 2)
    ).astype(int)

    df["TOTAL_EDUC_PADRES"] = df["F_EDUCACIONPADRE"] + df["F_EDUCACIONMADRE"]

    return df

In [45]:
# Aplicar la función de formateo
final_train_df = format_df(train_df)
final_test_df = format_df(test_df)

# Preparar X e y (recuerda que restamos 1 a y para que quede en [0,1,2,3])
X = final_train_df.drop(columns=["RENDIMIENTO_GLOBAL", "ID"], errors='ignore')
y = final_train_df["RENDIMIENTO_GLOBAL"].values - 1  # para que las clases sean [0, 1, 2, 3]

# Preparar X_test
X_test = final_test_df.drop(columns=["ID"], errors='ignore')

# Asegurar que no haya columnas tipo objeto
for df in [X, X_test]:
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = pd.factorize(df[col])[0]

# Confirmar dimensiones
print("X shape:", X.shape)
print("y shape:", y.shape)
print("X_test shape:", X_test.shape)

X shape: (692500, 56)
y shape: (692500,)
X_test shape: (296786, 56)


In [46]:
from xgboost import XGBClassifier

# Definir modelo con buenos hiperparámetros iniciales
xgb_model = XGBClassifier(
    objective='multi:softmax',
    num_class=4,
    learning_rate=0.1,
    max_depth=6,
    n_estimators=250,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='mlogloss',
    random_state=42
)

# Entrenar
xgb_model.fit(X, y)

# Predecir sobre el conjunto de prueba
y_pred = xgb_model.predict(X_test)

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [16:43:57] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [52]:
# Diccionario de clases
data_performance_dict = {
    0: 'bajo',
    1: 'medio-bajo',
    2: 'medio-alto',
    3: 'alto',
}

# Se convierte las predicciones numéricas a texto
y_labels = [data_performance_dict[val] for val in y_pred]

# Se recuperan los IDs reales del test
ids = test_df["ID"].values

# Se crea el DataFrame de salida
submission = pd.DataFrame({
    "ID": ids,
    "RENDIMIENTO_GLOBAL": y_labels
})

# Exportamos el archivo
submission.to_csv("out.csv", index=False)

In [48]:
from google.colab import files
files.download("out.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [53]:
!kaggle competitions submit \
  -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia \
  -f out.csv \
  -m "XGBoost modelo final"

100% 4.00M/4.00M [00:00<00:00, 12.6MB/s]
400 Client Error: Bad Request for url: https://www.kaggle.com/api/v1/competitions/submissions/submit/udea-ai-4-eng-20252-pruebas-saber-pro-colombia
